In [1]:
valid_inplace_types = (list, set)

inplace_slots = {
    "+=": "__iadd__",
    "-=": "__isub__",
    "*=": "__imul__",
    "/=": (1 / 2 == 0) and "__idiv__" or "__itruediv__",
    "//=": "__ifloordiv__",
    "%=": "__imod__",
    "**=": "__ipow__",
    "<<=": "__ilshift__",
    ">>=": "__irshift__",
    "&=": "__iand__",
    "^=": "__ixor__",
    "|=": "__ior__",
}

def __iadd__(x, y):
    x += y
    return x


def __isub__(x, y):
    x -= y
    return x


def __imul__(x, y):
    x *= y
    return x


def __idiv__(x, y):
    x /= y
    return x


def __ifloordiv__(x, y):
    x //= y
    return x


def __imod__(x, y):
    x %= y
    return x


def __ipow__(x, y):
    x **= y
    return x


def __ilshift__(x, y):
    x <<= y
    return x


def __irshift__(x, y):
    x >>= y
    return x


def __iand__(x, y):
    x &= y
    return x


def __ixor__(x, y):
    x ^= y
    return x


def __ior__(x, y):
    x |= y
    return x

inplace_ops = {
    "+=": __iadd__,
    "-=": __isub__,
    "*=": __imul__,
    "/=": __idiv__,
    "//=": __ifloordiv__,
    "%=": __imod__,
    "**=": __ipow__,
    "<<=": __ilshift__,
    ">>=": __irshift__,
    "&=": __iand__,
    "^=": __ixor__,
    "|=": __ior__,
}

In [2]:
def protected_inplacevar(op, var, expr):
    """Do an inplace operation

    If the var has an inplace slot, then disallow the operation
    unless the var an instance of ``valid_inplace_types``.
    """
    if hasattr(var, inplace_slots[op]) and not isinstance(var, valid_inplace_types):
        try:
            cls = var.__class__
        except AttributeError:
            cls = type(var)
        raise TypeError(
            "Augmented assignment to %s objects is not allowed"
            " in untrusted code" % cls.__name__
        )
    return inplace_ops[op](var, expr)

In [81]:
from RestrictedPython import compile_restricted
from RestrictedPython import safe_globals, safe_builtins, utility_builtins
from RestrictedPython import Eval, Guards, PrintCollector

In [123]:
import uuid

class PrintCollectorSingletonFactory:
    def create_singleton(self):
        instance = PrintCollector()
        
        # rewrite constructor
        def get_instance(self, _getattr_=None):
            instance._getattr_ = _getattr_
            return instance

        return type(f'PrintCollectorSingleton-{str(uuid.uuid4())}', (), {
            'instance': instance,
            'get_instance': get_instance
        })
        

ALLOWED_IMPORTS = frozenset([
    "array",
    "bisect",
    "calendar",
    "collections",
    "datetime",
    "heapq",
    "re",
    "string",
    "time",
    "typing",
    "zoneinfo"
])

def guarded_import(name, *args):
    return __import__(name, *args)
    # else:
    #    raise ImportError(f"{name} is not an allowed import.")

In [128]:
factory = PrintCollectorSingletonFactory()
singleton = factory.create_singleton()

allowed = {}
allowed['_getitem_'] = Eval.default_guarded_getitem
allowed['_getattr_'] = Eval.default_guarded_getattr
allowed['__metaclass__'] = type
allowed['__name__'] = 'name'
allowed['_getiter_'] = Eval.default_guarded_getiter
allowed['_iter_unpack_sequence_'] = Guards.guarded_iter_unpack_sequence
allowed['_print_'] = singleton.get_instance
allowed['_inplacevar_'] = protected_inplacevar
allowed['__builtins__'] = safe_builtins | {
    '__import__': guarded_import
}

In [125]:
bad_source = '''
def boom():
    arr = [1]
    return arr[2]
'''

In [129]:
loops = '''
import json
def boom():
    a = []
    for i in range(10):
        a.append(10-i)
    i = 0
    while i < 10:
        i+=1
    return sorted(a)
'''

In [130]:
byte_code = compile_restricted(loops, '<inline>', 'exec')
loc = {}
exec(byte_code, allowed , loc)
func = loc['boom']

In [109]:
func()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [110]:
singleton.instance.txt

["<module 'math' from '/home/nekozing/miniconda3/envs/testrunner/lib/python3.11/lib-dynload/math.cpython-311-x86_64-linux-gnu.so'>",
 '\n']